In [1]:
from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K


class MiniVGGNet:
    @staticmethod
    def build(width, height, depth, classes):
        model = Sequential()
        if K.image_data_format() == "channels_first":
            input_shape = (depth, height, width)
            chan_dim = 1
        else:
            input_shape = (height, width, depth)
            chan_dim = -1

        # first CONV => RELU => CONV => RELU => POOL layer set
        model.add(Conv2D(32, (3, 3), padding="same", input_shape=input_shape))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chan_dim))
        model.add(Conv2D(32, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chan_dim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        # second CONV => RELU => CONV => RELU => POOL layer set
        model.add(Conv2D(64, (3, 3), padding="same", input_shape=input_shape))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chan_dim))
        model.add(Conv2D(64, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chan_dim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        # First and only set of FC => RELU layers
        model.add(Flatten())
        model.add(Dense(512))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))

        # softmax classifier
        model.add(Dense(classes))
        model.add(Activation("softmax"))

        return model

In [2]:
from sklearn.preprocessing import LabelBinarizer
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD
from keras.datasets import cifar10

In [3]:
print("[INFO] loading CIFAR-10 data...")
((trainX, trainY), (testX, testY)) = cifar10.load_data()
trainX = trainX.astype("float") / 255.0
testX = testX.astype("float") / 255.0

[INFO] loading CIFAR-10 data...
170498071/170498071 [==============================] - 3s 0us/step


In [4]:
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.fit_transform(testY)

print("[INFO] compiling model...")
opt = SGD(learning_rate=0.01, decay=0.01 / 40, momentum=0.9, nesterov=True)
model = MiniVGGNet.build(width=32, height=32, depth=3, classes=10)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

[INFO] compiling model...


In [5]:
fname = "/content/weights/weights-{epoch:03d}-{val_loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(fname, monitor="val_loss", mode="min", save_best_only=True, verbose=1)
callbacks = [checkpoint]

print("[INFO] training network...")
H = model.fit(trainX, trainY, validation_data=(testX, testY), batch_size=64, epochs=40, callbacks=callbacks, verbose=1)

[INFO] training network...
Epoch 1/40
782/782 [==============================] - ETA: 0s - loss: 1.6723 - accuracy: 0.4401
Epoch 1: val_loss improved from inf to 1.43382, saving model to /content/weights/weights-001-1.4338.hdf5
782/782 [==============================] - 19s 13ms/step - loss: 1.6723 - accuracy: 0.4401 - val_loss: 1.4338 - val_accuracy: 0.5026
Epoch 2/40
780/782 [============================>.] - ETA: 0s - loss: 1.1641 - accuracy: 0.5936
Epoch 2: val_loss improved from 1.43382 to 1.13162, saving model to /content/weights/weights-002-1.1316.hdf5
782/782 [==============================] - 10s 12ms/step - loss: 1.1638 - accuracy: 0.5936 - val_loss: 1.1316 - val_accuracy: 0.6152
Epoch 3/40
778/782 [============================>.] - ETA: 0s - loss: 0.9576 - accuracy: 0.6678
Epoch 3: val_loss improved from 1.13162 to 0.94401, saving model to /content/weights/weights-003-0.9440.hdf5
782/782 [==============================] - 9s 11ms/step - loss: 0.9565 - accuracy: 0.6681 - val_